In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

In [3]:
# saving id of testset for future use
id = test_df.id

In [4]:
train_df.head()

,id,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
0,19717,Married-spouse-absent,Auto,Comedy,Male,45,Primetime,United-States,0.027465,High,No,False
1,31593,Married-civ-spouse,Pharma,Comedy,Male,45,Primetime,United-States,0.027465,Low,No,False
2,5681,Divorced,Entertainment,Comedy,Female,45,Primetime,United-States,0.027465,High,Yes,False
3,15491,Separated,Political,Infomercial,Female,40,Primetime,United-States,0.027465,Low,No,False
4,23587,Married-civ-spouse,Pharma,Comedy,Male,48,Primetime,United-States,0.027465,High,No,True


In [5]:
# checking for null values
train_df.isna().sum()

id                                   0
realtionship_status                  0
industry                             0
genre                                0
targeted_sex                         0
average_runtime(minutes_per_week)    0
airtime                              0
airlocation                          0
ratings                              0
expensive                            0
money_back_guarantee                 0
netgain                              0
dtype: int64

In [6]:
set(zip(train_df.airlocation.value_counts().sort_index().index,test_df.airlocation.value_counts().sort_index().index))

{('Cambodia', 'Cambodia'),
 ('Canada', 'Canada'),
 ('China', 'China'),
 ('Columbia', 'Columbia'),
 ('Cuba', 'Cuba'),
 ('Dominican-Republic', 'Dominican-Republic'),
 ('Ecuador', 'Ecuador'),
 ('El-Salvador', 'El-Salvador'),
 ('England', 'England'),
 ('France', 'France'),
 ('Germany', 'Germany'),
 ('Greece', 'Greece'),
 ('Guatemala', 'Guatemala'),
 ('Haiti', 'Haiti'),
 ('Holand-Netherlands', 'Honduras'),
 ('Honduras', 'Hong'),
 ('Hong', 'Hungary'),
 ('Hungary', 'India'),
 ('India', 'International'),
 ('International', 'Iran'),
 ('Iran', 'Ireland'),
 ('Ireland', 'Italy'),
 ('Italy', 'Jamaica'),
 ('Jamaica', 'Japan'),
 ('Japan', 'Laos'),
 ('Laos', 'Mexico'),
 ('Mexico', 'Nicaragua'),
 ('Nicaragua', 'Outlying-US(Guam-USVI-etc)'),
 ('Outlying-US(Guam-USVI-etc)', 'Peru'),
 ('Peru', 'Philippines'),
 ('Philippines', 'Poland'),
 ('Poland', 'Portugal'),
 ('Portugal', 'Puerto-Rico'),
 ('Puerto-Rico', 'Scotland'),
 ('Scotland', 'South'),
 ('South', 'Taiwan'),
 ('Taiwan', 'Thailand'),
 ('Thailand', '

From the above pairs of airlocation in training and testing set, we can observe that 'Holand-Netherlands' is missing in the testing set. While performing OneHotEncoder or Pandas' get dummies, the features in training and testing set will mismatch and create some overhead during prediction. So it is better to drop rows with airlocation as 'Holand-netherlands' in the training set

In [7]:
train_df.loc[train_df.airlocation=='Holand-Netherlands',:]

,id,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
8494,32363,Never-married,ClassAction,Comedy,Male,30,Morning,Holand-Netherlands,0.027465,Low,No,False
25764,17205,Never-married,ClassAction,Comedy,Female,24,Morning,Holand-Netherlands,0.027465,Low,No,False


There are only 2 rows with airlocation 'Holand-Netherlands' in the training set. Droping 2 rows from the training set doesn't affect that much. So, we are going to drop those rows using their id.

In [8]:
train_df = train_df.drop(train_df.index[[8494,25764]])

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
to_be_labeled_train = ['realtionship_status','industry','genre','targeted_sex','airtime','airlocation','expensive','money_back_guarantee','netgain']
for column in to_be_labeled_train:
    train_df[column] = le.fit_transform(train_df[column])

to_be_labeled_test = ['realtionship_status','industry','genre','targeted_sex','airtime','airlocation','expensive','money_back_guarantee']
for column in to_be_labeled_test:
    test_df[column] = le.fit_transform(test_df[column])

In [10]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
columns = ['average_runtime(minutes_per_week)','ratings']
for col in columns:
    train_df[col+'_scaled'] = scale.fit_transform(np.array(train_df[col]).reshape(-1, 1))
    test_df[col+'_scaled'] = scale.fit_transform(np.array(test_df[col]).reshape(-1, 1))

In [11]:
train_df.drop(columns,axis=1,inplace=True)
train_df = train_df.drop(['id'],axis=1)
test_df.drop(columns,axis=1,inplace=True)
test_df = test_df.drop(['id'],axis=1)

In [12]:
train_df

,realtionship_status,industry,genre,targeted_sex,airtime,airlocation,expensive,money_back_guarantee,netgain,average_runtime(minutes_per_week)_scaled,ratings_scaled
0,3,0,0,1,2,38,0,0,0,0.377019,-0.148336
1,2,4,0,1,2,38,1,0,0,0.377019,-0.148336
2,0,2,0,0,2,38,0,1,0,0.377019,-0.148336
3,5,5,3,0,2,38,1,0,0,-0.023650,-0.148336
4,2,4,0,1,2,38,0,0,1,0.617421,-0.148336
...,...,...,...,...,...,...,...,...,...,...,...
26043,2,4,0,1,2,38,1,0,1,0.777688,-0.148336
26044,4,0,0,0,1,38,1,0,0,-0.023650,-0.148336
26045,2,3,0,0,2,38,1,0,1,-1.225657,0.837376
26046,4,0,0,0,1,38,1,1,0,0.617421,-0.148336


In [13]:
train_df = pd.get_dummies(train_df,columns=['realtionship_status','industry','genre','targeted_sex','airtime','airlocation','expensive','money_back_guarantee'],drop_first=True)
test_df = pd.get_dummies(test_df,columns=['realtionship_status','industry','genre','targeted_sex','airtime','airlocation','expensive','money_back_guarantee'],drop_first=True)

In [14]:
train_df

,netgain,average_runtime(minutes_per_week)_scaled,ratings_scaled,realtionship_status_1,realtionship_status_2,realtionship_status_3,realtionship_status_4,realtionship_status_5,realtionship_status_6,industry_1,...,airlocation_34,airlocation_35,airlocation_36,airlocation_37,airlocation_38,airlocation_39,airlocation_40,expensive_1,expensive_2,money_back_guarantee_1
0,0,0.377019,-0.148336,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0.377019,-0.148336,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,0.377019,-0.148336,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0,-0.023650,-0.148336,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,0.617421,-0.148336,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,1,0.777688,-0.148336,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
26044,0,-0.023650,-0.148336,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
26045,1,-1.225657,0.837376,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
26046,0,0.617421,-0.148336,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,1


In [15]:
X = train_df.drop(['netgain'],axis=1)
y = train_df.netgain

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='lbfgs',max_iter=1000)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
accuracy_score(y_pred,y_test)

0.8035132619823173

In [19]:
from sklearn.neighbors import KNeighborsClassifier
scores = []
for i in range(1,30):
  knn = KNeighborsClassifier(n_neighbors=i)
  knn.fit(X_train,y_train)
  y_pred = knn.predict(X_test)
  scores.append(accuracy_score(y_pred,y_test))


In [20]:
for i,j in enumerate(scores):
  if j == np.array(scores).max():
    print(i+1,j)

19 0.809097254536994


In [21]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 1000, random_state = 0)
rf_classifier.fit(X_train,y_train)
y_pred = rf_classifier.predict(X_test)
accuracy_score(y_pred,y_test)

0.8065379246161005

In [22]:
from sklearn.svm import SVC
svc_scores = []
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
for i in range(len(kernels)):
    svc_classifier = SVC(kernel = kernels[i])
    svc_classifier.fit(X_train,y_train)
    y_pred = svc_classifier.predict(X_test)
    svc_scores.append(accuracy_score(y_pred,y_test))

In [23]:
svc_scores

[0.8056072591903211, 0.809097254536994, 0.8134015821312238, 0.7390646812470917]

In [24]:
# logreg = LogisticRegression(solver='lbfgs',max_iter=1000)
# logreg.fit(X,y)
# y_pred = knn.predict(test_df)
# final_df = pd.DataFrame({'id': id, 'netgain': y_pred})
# final_df['netgain'] = final_df['netgain'].map({0:False,1:True})
# final_df.to_csv('logreg_Results.csv',index=False)

# knn = KNeighborsClassifier(n_neighbors=19)
# knn.fit(X,y)
# y_pred = knn.predict(test_df)
# final_df = pd.DataFrame({'id': id, 'netgain': y_pred})
# final_df['netgain'] = final_df['netgain'].map({0:False,1:True})
# final_df.to_csv('knn_Results.csv',index=False)


# rf_classifier = RandomForestClassifier(n_estimators = 1000, random_state = 0)
# rf_classifier.fit(X,y)
# y_pred = rf_classifier.predict(test_df)
# final_df = pd.DataFrame({'id': id, 'netgain': y_pred})
# final_df['netgain'] = final_df['netgain'].map({0:False,1:True})
# final_df.to_csv('rf_Results.csv',index=False)

svc_classifier = SVC(kernel = 'rbf')
svc_classifier.fit(X,y)
y_pred = svc_classifier.predict(test_df)
final_df = pd.DataFrame({'id': id, 'netgain': y_pred})
final_df['netgain'] = final_df['netgain'].map({0:False,1:True})
final_df.to_csv('svc_Results.csv',index=False)